---
title: Parallax / Ptychography Alignment Notebook
authors: [gvarnavides]
date: 2025-06-11
---

In [1]:
%matplotlib widget

import numpy as np
import matplotlib.pyplot as plt
import h5py
from matplotlib.gridspec import GridSpec
import ctf
import py4DSTEM

from IPython.display import display
import ipywidgets
plt.rcParams['text.color']='white'
plt.rcParams['xtick.labelcolor'] = 'white'
plt.rcParams['xtick.color'] = 'white'
plt.rcParams['ytick.labelcolor'] = 'white'
plt.rcParams['ytick.color'] = 'white'
plt.rcParams['axes.labelcolor'] = 'white'
plt.rcParams['axes.edgecolor'] = 'white'

In [2]:
with h5py.File("data/parallax-ptycho-demo-data.h5",'r') as f:
    pre_vbf_fourier = f['pre_vbf_fourier'][:]
    pre_vbf_fourier_noisy = f['pre_vbf_fourier_noisy'][:]
    pre_grad_k = f['pre_grad_k'][:]
    pre_kxy = f['pre_kxy'][:]
with h5py.File("data/parallax-ptycho-demo-op_shift.h5",'r') as f:
    pre_op_shift = f['pre_op_shift'][:]
with h5py.File("data/parallax-ptycho-demo-op_par.h5",'r') as f:
    pre_op_par = f['pre_op_par'][:]
with h5py.File("data/parallax-ptycho-demo-op_full.h5",'r') as f:
    pre_op_full = f['pre_op_full'][:]

In [3]:
upsampling_factor=2
def bin_array(
    array,
    bin_factor=upsampling_factor
):
    sx,sy = array.shape[-2:]
    return array.reshape(
        (
            -1,
            sx//bin_factor,
            bin_factor,
            sy//bin_factor,
            bin_factor,
        )
    ).sum((-3,-1))

tiled_vbfs = np.tile(
    pre_vbf_fourier,
    (1,upsampling_factor,upsampling_factor)
)

tiled_vbfs_noisy = np.tile(
    pre_vbf_fourier_noisy,
    (1,upsampling_factor,upsampling_factor)
)
unshifted_stack = bin_array(np.fft.ifft2(tiled_vbfs)).real
shifted_stack = bin_array(np.fft.ifft2(tiled_vbfs* pre_op_shift)).real
parallax_stack = bin_array(np.fft.ifft2(tiled_vbfs* pre_op_par)).real
ptycho_stack = bin_array(np.fft.ifft2(tiled_vbfs* pre_op_full)).imag

unshifted_stack_noisy = bin_array(np.fft.ifft2(tiled_vbfs_noisy)).real
shifted_stack_noisy = bin_array(np.fft.ifft2(tiled_vbfs_noisy* pre_op_shift)).real
parallax_stack_noisy = bin_array(np.fft.ifft2(tiled_vbfs_noisy* pre_op_par)).real
ptycho_stack_noisy = bin_array(np.fft.ifft2(tiled_vbfs_noisy* pre_op_full)).imag

In [4]:
index = 0

In [5]:
dpi = 72
with plt.ioff():
    fig = plt.figure(figsize=(625/dpi,205/dpi),dpi=dpi)
gs = GridSpec(2, 7, figure=fig,hspace=0,wspace=0)

ax_l = fig.add_subplot(gs[:, :2])
ax_c_01 = fig.add_subplot(gs[0, 2])
ax_c_02 = fig.add_subplot(gs[0, 3])
ax_c_03 = fig.add_subplot(gs[0, 4])
ax_c_11 = fig.add_subplot(gs[1, 2])
ax_c_12 = fig.add_subplot(gs[1, 3])
ax_c_13 = fig.add_subplot(gs[1, 4])
ax_r = fig.add_subplot(gs[:, 5:])

arrows = pre_grad_k.copy()
arrows[index:]=0

ax_l.patch.set_alpha(0)
ax_l.quiver(
    pre_kxy[:,1],
    pre_kxy[:,0],
    arrows[:,1],
    arrows[:,0],
    angles='xy',
    scale=7e3,
)
scatter = ax_l.collections[0]
scatter.set_color((255/255, 249/255, 148/255))

dot = ax_l.scatter(
    pre_kxy[index,1],
    -pre_kxy[index,0],
    color='white'
)
ax_l.set(xlim=[-0.25,0.25],ylim=[-0.25,0.25],xticks=[],yticks=[],aspect='equal')
ax_l.set_title("cross-correlation shifts")

inf_unshifted_scaled, _, _ = py4DSTEM.visualize.return_scaled_histogram_ordering(unshifted_stack[index],normalize=True)
im_inf_unshifted = ax_c_01.imshow(inf_unshifted_scaled,cmap='gray')
ax_c_01.set_title("vBF",fontsize=12)

fin_unshifted_scaled, _, _ = py4DSTEM.visualize.return_scaled_histogram_ordering(unshifted_stack_noisy[index],normalize=True)
im_fin_unshifted = ax_c_02.imshow(fin_unshifted_scaled,cmap='gray')
ax_c_02.set_title("noisy vBF",fontsize=12)

ibf_unshifted_scaled, _, _ = py4DSTEM.visualize.return_scaled_histogram_ordering(unshifted_stack_noisy[:index+1].mean(0),normalize=True)
im_ibf_unshifted = ax_c_03.imshow(ibf_unshifted_scaled,cmap='gray')
ax_c_03.set_title("incoherent BF",fontsize=12)

inf_par_scaled, _, _ = py4DSTEM.visualize.return_scaled_histogram_ordering(shifted_stack[index],normalize=True)
im_inf_par = ax_c_11.imshow(inf_par_scaled,cmap='gray')
ax_c_11.set_xlabel("deconv. vBF",fontsize=12)

fin_par_scaled, _, _ = py4DSTEM.visualize.return_scaled_histogram_ordering(shifted_stack_noisy[index],normalize=True)
im_fin_par = ax_c_12.imshow(fin_par_scaled,cmap='gray')
ax_c_12.set_xlabel("noisy deconv.",fontsize=12)

ibf_par_scaled, _, _ = py4DSTEM.visualize.return_scaled_histogram_ordering(shifted_stack_noisy[:index+1].mean(0),normalize=True)
im_ibf_par = ax_c_13.imshow(ibf_par_scaled,cmap='gray')
ax_c_13.set_xlabel("coherent BF",fontsize=12)

ax_c_03.set_visible(False)
ax_c_13.set_visible(False)

im_operator = ax_r.imshow(
    ctf.complex_to_rgb(
       np.fft.fftshift(
           1j* pre_op_shift[index]
       ),
        vmin=0,
        vmax=1
    )
)
ax_r.set(xticks=[],yticks=[],aspect='equal',title='deconvolution operator')

bar = {
    'pixelsize':0.35,
    'pixelunits':'nm',
    "Nx":72,
    "Ny":72,
    "labelsize":8,
    "width":2,
    "length":5
}

for ax in [ax_c_01,ax_c_02,ax_c_03,ax_c_11,ax_c_12,ax_c_13]:
    py4DSTEM.visualize.add_scalebar(ax,bar)
    ax.set(xticks=[],yticks=[],aspect='equal')

gs.tight_layout(fig)
fig.patch.set_alpha(0)
fig.canvas.resizable = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
fig.canvas.toolbar_visible = False
fig.canvas.layout.width = '625px'
fig.canvas.layout.height = "225px"
fig.canvas.toolbar_position = "bottom"
None

In [11]:
def update_scatter_plot(
    index,
):
    """ """
    arrows = pre_grad_k.copy()
    arrows[index:]=0
    
    scatter.set_UVC(arrows[:,1],arrows[:,0])
    dot.set_offsets([-pre_kxy[index,1],pre_kxy[index,0]])
                       
    fig.canvas.draw_idle()
    return None

def update__plots(
    index,
):
    """ """
    
    inf_unshifted_scaled, _, _ = py4DSTEM.visualize.return_scaled_histogram_ordering(unshifted_stack[index],normalize=True)
    im_inf_unshifted.set_data(inf_unshifted_scaled)
    
    fin_unshifted_scaled, _, _ = py4DSTEM.visualize.return_scaled_histogram_ordering(unshifted_stack_noisy[index],normalize=True)
    im_fin_unshifted.set_data(fin_unshifted_scaled)

    ibf_unshifted_scaled, _, _ = py4DSTEM.visualize.return_scaled_histogram_ordering(unshifted_stack_noisy[:index+1].mean(0),normalize=True)
    im_ibf_unshifted.set_data(ibf_unshifted_scaled)

    if mode.value == "deconv: parallax":
        inf_par_scaled, _, _ = py4DSTEM.visualize.return_scaled_histogram_ordering(shifted_stack[index],normalize=True)
        fin_par_scaled, _, _ = py4DSTEM.visualize.return_scaled_histogram_ordering(shifted_stack_noisy[index],normalize=True)
        ibf_par_scaled, _, _ = py4DSTEM.visualize.return_scaled_histogram_ordering(shifted_stack_noisy[:index+1].mean(0),normalize=True)
        operator = ctf.complex_to_rgb(
           np.fft.fftshift(
               1j* pre_op_shift[index]
           ),
            vmin=0,
            vmax=1
        )
    elif mode.value == "deconv: phase flip":
        inf_par_scaled, _, _ = py4DSTEM.visualize.return_scaled_histogram_ordering(parallax_stack[index],normalize=True)
        fin_par_scaled, _, _ = py4DSTEM.visualize.return_scaled_histogram_ordering(parallax_stack_noisy[index],normalize=True)
        ibf_par_scaled, _, _ = py4DSTEM.visualize.return_scaled_histogram_ordering(parallax_stack_noisy[:index+1].mean(0),normalize=True)
        operator = ctf.complex_to_rgb(
           np.fft.fftshift(
               1j* pre_op_par[index]
           ),
            vmin=0,
            vmax=1
        )
    elif mode.value == "deconv: full":
        inf_par_scaled, _, _ = py4DSTEM.visualize.return_scaled_histogram_ordering(ptycho_stack[index],normalize=True)
        fin_par_scaled, _, _ = py4DSTEM.visualize.return_scaled_histogram_ordering(ptycho_stack_noisy[index],normalize=True)
        ibf_par_scaled, _, _ = py4DSTEM.visualize.return_scaled_histogram_ordering(ptycho_stack_noisy[:index+1].mean(0),normalize=True)
        operator = ctf.complex_to_rgb(
           np.fft.fftshift(
               pre_op_full[index]
           ),
            vmin=0,
            vmax=1
        )
        
    im_inf_par.set_data(inf_par_scaled)
    im_fin_par.set_data(fin_par_scaled)
    im_ibf_par.set_data(ibf_par_scaled)
    im_operator.set_data(operator)
        
    fig.canvas.draw_idle()
    return None

def update_both(change):
    """ """
    index = change["new"]
    update_scatter_plot(index)
    update__plots(index)
    return None

def toggle_visibility(change):
    """ """
    visibility = change["new"]
    ax_c_03.set_visible(visibility)
    ax_c_13.set_visible(visibility)
    fig.canvas.draw_idle()
    return None

def change_mode(change):
    """ """
    update_both({"new":slider.value})
    fig.canvas.draw_idle()
    return None

layout = ipywidgets.Layout(width='600px',height='30px')
half_layout = ipywidgets.Layout(width='300px',height='30px')
quarter_layout = ipywidgets.Layout(width='150px',height='30px')

style = {
    'description_width': 'initial',
}

toggle = ipywidgets.ToggleButton(
    value=False,
    description="align images",
    style=style,
    layout=quarter_layout,
)

play = ipywidgets.Play(
    value=0,
    min=0,
    max=len(pre_kxy)-1,
    step=1,
    interval=250,
    show_repeat=False,
    style=style,
    layout=quarter_layout,
)

mode = ipywidgets.Dropdown(
    options=['deconv: parallax', 'deconv: phase flip', 'deconv: full'],
    # description='deconvolution:',
    style=style,
    layout=quarter_layout,
)

slider = ipywidgets.IntSlider(
    min=0,
    max=len(pre_kxy)-1,
    step=1,
    layout=half_layout,
    style=style,
    description="planewave index"
)

ipywidgets.jslink((play, 'value'), (slider, 'value'))
slider.observe(update_both,"value")
toggle.observe(toggle_visibility,"value")
mode.observe(change_mode,"value")

In [12]:
#| label: app:parallax-alignment

ipywidgets.VBox(
    [
        ipywidgets.HBox([slider,toggle,mode],layout=layout),
        fig.canvas
    ],
    layout=ipywidgets.Layout(
        align_items="center"
    )
)